In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.transform import pyramid_gaussian, resize

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [3]:
# --- 1. SYNTHETIC DATA GENERATION ---
# We will create a 100x100 grid to represent our world.
# Each pixel is a "grid cell" as described in your paper.
MAP_SIZE = (100, 100)

In [4]:
def create_synthetic_data(size):
    """
    Generates synthetic geospatial maps for our features and target.
    This simulates the data you would source from GEE, WDPA, etc.
    """
    print(f"Creating synthetic {size} maps...")
    
    # --- Target Variable (y) ---
    # This is our "World Database on Protected Areas (WDPA)" layer
    # 0 = Not Protected, 1 = Protected
    existing_pas = np.zeros(size)
    # Create a 30x30 "National Park" in a high-value area
    existing_pas[10:40, 10:40] = 1
    # Create a smaller 15x15 "Reserve"
    existing_pas[70:85, 10:25] = 1

    # --- Feature Variables (X) ---
    
    # Feature 1: Environmental Value (like NDVI)
    # Hypothesis: PAs are MORE likely in high-value areas.
    env_value_map = np.zeros(size)
    # The "National Park" area is a high-value hotspot
    env_value_map[5:45, 5:45] = 0.9
    # The "Reserve" area is also high-value
    env_value_map[65:90, 5:30] = 0.8
    # Add another "unprotected" hotspot (where transition risk should be high)
    env_value_map[20:40, 70:90] = 0.85
    
    # Feature 2: Economic Activity (like GDP / Night Lights)
    # Hypothesis: PAs are LESS likely in high-GDP areas (high opportunity cost).
    gdp_map = np.zeros(size)
    # Create a 25x25 "high-GDP city" where protection is unlikely
    gdp_map[50:75, 60:85] = 1.0 
    # Add some noise
    gdp_map += np.random.rand(*size) * 0.1
    
    # Feature 3: Population Density
    # Hypothesis: PAs are LESS likely in high-population areas.
    pop_map = np.zeros(size)
    # Population is high near the city
    pop_map[45:80, 55:90] = 0.8
    # Add some "villages"
    pop_map[15:25, 50:60] = 0.3
    pop_map += np.random.rand(*size) * 0.05
    
    # Return a dictionary of feature maps and the single target map
    feature_maps = {
        'env_value': env_value_map,
        'gdp': gdp_map,
        'population': pop_map
    }
    
    return feature_maps, existing_pas

In [ ]:
# --- 2. PYRAMID FEATURE ENGINEERING ---

def build_pyramid_features(feature_maps, target_map):
    """
    Takes the raw maps and engineers multi-scale features using a
    Gaussian Pyramid. This is the core of the "pyramid representation".
    """
    print("Engineering pyramid features...")
    
    # This will hold all our features, one row per pixel
    features_list = []
    
    # Flatten the target map to be our 'y' variable
    # .ravel() turns a 2D map into a 1D list
    target_series = pd.Series(target_map.ravel(), name="is_protected")
    
    for name, map_data in feature_maps.items():
        # ---
        # Add the original map (Scale 0)
        # This is the pixel's own value
        # ---
        features_list.append(
            pd.Series(map_data.ravel(), name=f"{name}_scale_0")
        )
        
        # ---
        # Build a Gaussian Pyramid
        # This creates blurred, downsampled versions of the map.
        # We use `downscale=4` for more distinct scales.
        # max_layer=3 gives us 3 pyramid levels (plus scale 0).
        # ---
        pyramid = list(pyramid_gaussian(map_data, downscale=4, max_layer=3))
        
        # pyramid[0] is the original 100x100
        # pyramid[1] is 25x25 (blurred average of 4x4 blocks)
        # pyramid[2] is 7x7 (blurred average of 16x16 blocks)
        # pyramid[3] is 2x2 (blurred average of 64x64 blocks)

        # Loop through the smaller pyramid layers (starting from layer 1)
        for i, layer in enumerate(pyramid[1:], 1):
            
            # ---
            # Now we resize the small, blurred layer back up to the
            # original 100x100 size.
            # ---
            # This means every pixel now has new features:
            # - Its original value (scale 0)
            # - The average value of its 4x4 neighborhood (scale 1)
            # - The average value of its 16x16 neighborhood (scale 2)
            # - etc.
            scaled_layer = resize(
                layer, 
                MAP_SIZE, 
                anti_aliasing=True, 
                preserve_range=True
            )
            
            # Add this new multi-scale feature to our list
            features_list.append(
                pd.Series(scaled_layer.ravel(), name=f"{name}_scale_{i}")
            )
            
    # Combine all feature lists into a single DataFrame
    X = pd.concat(features_list, axis=1)
    y = target_series
    
    return X, y

